In [2]:
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

### Preprocessing data

In [3]:
def insertDataKetinggalan(dataFrame : pd.DataFrame,
                          pasalBefore : list,
                          namePasal : list,
                          pasalChange : list) -> pd.DataFrame :
    
    for before,name,pasal in zip(pasalBefore,namePasal,pasalChange):
        indexBefore = dataFrame[dataFrame["Pasal"] ==  before].index[0] + 0.5
        dataFrame.loc[indexBefore] = [name,pasal]
        dataFrame = dataFrame.sort_index().reset_index(drop=True)
    return dataFrame

def lihatPasal(dataFrame : pd.DataFrame, pasal : str | int) -> pd.DataFrame:
    return dataFrame[dataFrame["Pasal"] == f"Pasal {pasal}"]

In [4]:
# pasal ketinggalan
pasal331 = "Dalam setiap perwalian, hanya ada seorang wali, kecuali yang ditentukan dalam Pasal 351 dan 361. Perwalian untuk anak-anak dari bapak dan ibu yang sama, harus dipandang sebagai suatu perwalian, sejauh anak-anak itu mempunyai seorang wali yang sama. "
pasal319m = "Segala surat-surat permohonan, tuntutan, penetapan. pemberitahuan dan semua surat lain yang dibuat untuk memenuhi ketentuan-ketentuan dalam bagian ini, bebas dan materai. Segala permohonan termaksud dalam bagian ini, yang diajukan oleh dewan perwalian, harus diperiksa oleh Pengadilan Negeri dengan cuma-cuma, dan salinan-salinan yang diminta oleh dewan-dewan itu untuk salinan-salinan yang diperintahkan kepadanya, harus diberikan oleh panitera kepada mereka secara bebas dan segala biaya. "

In [5]:
pasal365 = """Dalam segala hal, bila Hakim harus mengangkat seorang wali, maka perwalian itu boleh
diperintahkan kepada perkumpulan berbadan hukum yang berkedudukan di Indonesia, kepada
suatu yayasan atau kepada lembaga sosial yang berkedudukan di Indonesia yang menurut
anggaran dasarnya, akta pendiriannya atau reglemennya mengatur pemeliharaan anak belum
dewasa untuk waktu yang lama. Pasal 362 tidak berlaku. Perkumpulan, yayasan atau lembaga sosial itu, sehubungan dengan perwalian yang ditugaskan
kepadanya, mempunyai hak-hak dan kewajiban-kewajiban yang sama dengan yang diberikan atau
yang diperintahkan kepada wali, kecuali jika undang-undang menentukan lain.
Para anggota pengurus masing-masing bertanggung jawab secara pribadi dan tanggungmenanggung atas pelaksanaan perwalian itu, selama perwalian itu dilakukan oleh pengurus dan
selama anggota-anggota pengurus ini tidak menunjukkan pada Hakim, bahwa mereka telah
mencurahkan segala usaha guna melaksanakan perwalian sebagaimana mestinya atau mereka
dalam keadaan tidak mampu menjaganya. Pengurus boleh memberi kuasa secara tertulis kepada seorang anggotanya atau lebih untuk
melakukan perwalian terhadap anak-anak yang belum dewasa tersebut dalam surat kuasa itu.
Pengurus berhak pula atas kehendaknya menyerahkan pengurusan harta kekayaan anak-anak
belum dewasa yang dengan tegas ditunjuknya, asalkan secara tertulis, kepada Balai Harta
Peninggalan, yang dengan demikian wajib menerima pengurusan itu dan menyelenggarakannya
menurut ketentuan-ketentuan yang berlaku terhadapnya. Penyerahan ini tidak dapat dicabut. 
"""

In [6]:
base_data = pd.read_csv("./new_clean_csv.csv")
base_data.drop(columns=["Unnamed: 0"],inplace=True)

In [7]:
base_data = insertDataKetinggalan(base_data,
                            ["Pasal 319k","Pasal 330","Pasal 364"],
                            ["Pasal 319m","Pasal 331","Pasal 365"],
                            [pasal319m,pasal331,pasal365])

In [9]:
base_data.to_csv("new_clean_usage.csv",index=False)

#### Fungsi memasukan dataset bagian 

In [14]:
def insertBabToData(dataFrame : pd.DataFrame, rangeBab : list, nameBab : str, returnDf : bool = False) -> None | pd.DataFrame:
    rangeBab = [f"Pasal {a}" for a in rangeBab]
    startLocation = dataFrame[dataFrame["Pasal"] == rangeBab[0]].index[0]
    endLocation = dataFrame[dataFrame["Pasal"] == rangeBab[1]].index[0]
    
    # memasukan data bab
    dataFrame.loc[startLocation:endLocation,"Bab"] = nameBab
    dataFrame["Bab"].astype("str")

    if(returnDf):
        return dataFrame
     

    

### Fungsi Memasukan bagian kedalam Data

In [15]:
def insertBagianToData(dataFrame : pd.DataFrame, rangeBab : list, nameBab : str, returnDf : bool = False) -> None | pd.DataFrame:
    rangeBab = [f"Pasal {a}" for a in rangeBab]
    startLocation = dataFrame[dataFrame["Pasal"] == rangeBab[0]].index[0]
    if len(rangeBab) == 1:
        endLocation = dataFrame[dataFrame["Pasal"] == rangeBab[0]].index[0]
    else:
        endLocation = dataFrame[dataFrame["Pasal"] == rangeBab[1]].index[0]
    
    # memasukan data bagian
    dataFrame.loc[startLocation:endLocation,"Bagian"] = nameBab
    dataFrame["Bagian"].astype("str")

    if(returnDf):
        return dataFrame
    

### Fungsi Iterasi Memasukan Data

In [16]:
def iterationInsert(status : str, dfTarget : pd.DataFrame, dfChange : pd.DataFrame,func : object) -> None:
    title =  dfChange[status.title()].tolist()
    rangePasal = [a.split("-") for a in dfChange["Range Pasal"]]

    for t,r in tqdm(zip(title,rangePasal),desc=f"Inserting {status}"):
        try :
            func(dfTarget,r,t)
        except :
            print(f"ada kesalahan di {r}")
            break
        print(f"\r perulangan ke {r}",end=" ")


### Buku 1

In [41]:
book1 = base_data.loc[:base_data[base_data["Pasal"] == "Pasal 495"].index[0]]

rangeBab = pd.read_csv("./range-pasal/Data Range  Pasal - Buku 1 -Bab.csv")
rangeBab["Bab"] = rangeBab["Bab"].apply(lambda x : x.title())

#### Memasukan Bab

In [42]:
iterationInsert("Bab",book1,rangeBab,insertBabToData)

Inserting Bab: 0it [00:00, ?it/s]

 perulangan ke ['463', '495']  

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1400\2454618736.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame.loc[startLocation:endLocation,"Bab"] = nameBab
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1400\2454618736.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Kitab Undang-Undang Hukum Perdata' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataFrame.loc[startLocation:endLocation,"Bab"] = nameBab


#### Memasukan Bagian

In [43]:
rangeBagian = pd.read_csv("./range-bagian/Data Range  Pasal - Buku 1 - Bagian.csv")
rangeBagian.drop(columns=["No"],inplace=True)
rangeBagian["Bagian"] = rangeBagian["Bagian"].apply(lambda x : x.title())

In [44]:
iterationInsert("Bagian",book1,rangeBagian,insertBagianToData)

Inserting Bagian: 0it [00:00, ?it/s]

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1400\3510560759.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame.loc[startLocation:endLocation,"Bagian"] = nameBab
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1400\3510560759.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Daftar Catatan Sipil Pada Umumnya' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataFrame.loc[startLocation:endLocation,"Bagian"] = nameBab


 perulangan ke ['493', '495']  

In [51]:
book1["Bagian"].fillna("Tidak terdapat Dalam Bagian",inplace=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1400\2451261974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book1["Bagian"].fillna("Tidak terdapat Dalam Bagian",inplace=True)


In [55]:
book1

,Pasal,Isi,Bab,Bagian
0,Pasal 1,Menikmati hak-hak kewargaan tidak tergantung p...,Kitab Undang-Undang Hukum Perdata,Tidak terdapat Dalam Bagian
1,Pasal 2,Anak yang ada dalam kandungan seorang perempua...,Kitab Undang-Undang Hukum Perdata,Tidak terdapat Dalam Bagian
2,Pasal 3,Tiada suatu hukuman pun yang mengakibatkan kem...,Kitab Undang-Undang Hukum Perdata,Tidak terdapat Dalam Bagian
3,Pasal 4,Tanpa mengurangi ketentuan dalam Pasal 10 Kete...,Akta-Akta Catatan Sipil,Daftar Catatan Sipil Pada Umumnya
4,Pasal 5,"Presiden, setelah mendengar Mahkamah Agung men...",Acara Yang Harus Mendahului Perkawinan,Daftar Catatan Sipil Pada Umumnya
...,...,...,...,...
517,Pasal 491,Ketentuan-ketentuan dari kedua pasal yang lalu...,Ketidakhadiran,Hak-Hak Yang Jatuh Ke Tangan Orang Tak Hadir Y...
518,Pasal 492,Bila kemudian orang yang dalam keadaan tak had...,Ketidakhadiran,Hak-Hak Yang Jatuh Ke Tangan Orang Tak Hadir Y...
519,Pasal 493,"Bila salah seorang dari suami isteri, selain m...",Ketidakhadiran,Akibat-Akibat Keadaan Tidak Hadir Berkenaan De...
520,Pasal 494,"Bila atas panggilan ketiga dari Pengadilan, ba...",Ketidakhadiran,Akibat-Akibat Keadaan Tidak Hadir Berkenaan De...


In [56]:
book1.describe()

,Pasal,Isi,Bab,Bagian
count,522,522,521,522
unique,522,522,17,46
top,Pasal 1,Menikmati hak-hak kewargaan tidak tergantung p...,Acara Yang Harus Mendahului Perkawinan,Tidak terdapat Dalam Bagian
freq,1,1,114,121


In [60]:
book1.to_csv("./clean_book/book 1.csv",index=False)

NameError: name 'base_data' is not defined